In [1]:
from keras.datasets import imdb
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras import layers

In [2]:
data = pd.read_csv('IMDB Dataset.csv')
data['review'] = data['review'].str.lower()
data.tail()

,review,sentiment
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative
49999,no one expects the star trek movies to be high...,negative


In [9]:
data.iloc[1,:]

review       a wonderful little production. <br /><br />the...
sentiment                                             positive
Name: 1, dtype: object

In [3]:
data.shape

(50000, 2)

In [4]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [5]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

/tmp/ipykernel_49/2493842820.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [6]:
reviews_list = []
for i in range(len(data.review)): 
  reviews_list.append(data.review[i])
  sentiment = data_without_stopwords['sentiment']

In [7]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [8]:
Y_train

array([1, 0, 0, ..., 1, 0, 0])

In [9]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [10]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [11]:
word_to_vec_map = read_glove_vector('glove.6B.50d.txt')

maxLen = 150

In [12]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [13]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)
  print(X_indices)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings) # First Layer

  X = Dropout(0.15)(X)

  X = LSTM(128, return_sequences=True)(X) # Second Layer

  X = Dropout(0.15)(X)

  X = LSTM(128)(X) # Third layer

  X = Dense(1, activation='sigmoid')(X)

  return  Model(inputs=X_indices, outputs=X)


In [14]:
m= imdb_rating(150)
m

KerasTensor(type_spec=TensorSpec(shape=(None, 150), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [15]:
X_train_indices = tokenizer.texts_to_sequences(X_train)
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [16]:
X_train_indices

array([[ 167, 1910,    3, ...,    3,  955,   97],
       [3633,    7,    7, ...,  608,  618,    1],
       [  34,  905,    9, ...,  488,  488,   19],
       ...,
       [   6,  993,   14, ...,  542, 2438, 3380],
       [  10,   37, 1240, ...,    0,    0,    0],
       [  77,  206,  135, ...,   51, 3362, 1471]], dtype=int32)

In [17]:
#adam = Adam(learning_rate = 0.0001)
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)

m.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
m.fit(X_train_indices, Y_train,batch_size=64, epochs=21)

Epoch 1/21
625/625 [==============================] - 21s 26ms/step - loss: 0.5880 - accuracy: 0.6838
Epoch 2/21
625/625 [==============================] - 16s 26ms/step - loss: 0.5065 - accuracy: 0.7534
Epoch 3/21
625/625 [==============================] - 16s 26ms/step - loss: 0.4707 - accuracy: 0.7781
Epoch 4/21
625/625 [==============================] - 16s 26ms/step - loss: 0.4550 - accuracy: 0.7858
Epoch 5/21
625/625 [==============================] - 16s 26ms/step - loss: 0.4313 - accuracy: 0.8014
Epoch 6/21
625/625 [==============================] - 16s 26ms/step - loss: 0.4163 - accuracy: 0.8106
Epoch 7/21
625/625 [==============================] - 16s 26ms/step - loss: 0.4011 - accuracy: 0.8178
Epoch 8/21
625/625 [==============================] - 16s 26ms/step - loss: 0.3930 - accuracy: 0.8232
Epoch 9/21
625/625 [==============================] - 16s 26ms/step - loss: 0.3800 - accuracy: 0.8295
Epoch 10/21
625/625 [==============================] - 16s 26ms/step - loss: 0.373

In [18]:
!mkdir -p saved_model
m.save('saved_model/my_model') 

2022-02-01 20:31:16.325908: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [19]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           5612100   
                                                                 
 lstm (LSTM)                 (None, 150, 128)          91648     
                                                                 
 dropout (Dropout)           (None, 150, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 150, 128)          131584    
                                                                 
 dropout_1 (Dropout)         (None, 150, 128)          0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584

In [20]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [21]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
m.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 5s 12ms/step - loss: 0.3507 - accuracy: 0.8441


[0.3506869375705719, 0.8440999984741211]

In [22]:
loss, acc = new_model.evaluate(X_test_indices, Y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(X_test_indices).shape)

313/313 - 4s - loss: 0.3507 - accuracy: 0.8441 - 4s/epoch - 13ms/step
Restored model, accuracy: 84.41%
(10000, 1)


In [23]:
preds = m.predict(X_test_indices)

n = np.random.randint(0,9999)

X_test[n]

if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

precicted sentiment : negative
correct sentiment : negative


In [24]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = m.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

In [25]:
a = []
for i in range(0,50000):
    a+=[[i]]

In [26]:
new_data = add_score_predictions(data,a)
new_data.head()

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.541370,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.492336,negative
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.614129,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.529855,positive
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.495821,negative


In [27]:
data.review[1]

'a wonderful little production. <br /><br />the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. <br /><br />the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well d